<a href="https://colab.research.google.com/github/snow-The/GW190521/blob/main/homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 依賴安裝
!pip install "ml4gw>=0.7.10" "gwpy>=3.0" "h5py>=3.12" "torchmetrics>=1.6" "lightning>=2.4.0" "rich>=10.2.2,<14.0"

In [ ]:
import torch
import numpy as np
from gwpy.timeseries import TimeSeries
from ml4gw.transforms import Whiten, SpectralDensity

# 1. 設定設備
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 2. 設定參數
trigger_time = 1242442967.4
sample_rate = 2048
start_time = trigger_time - 6
end_time = trigger_time + 2
# 分析總時長 = 8 秒
raw_duration = end_time - start_time

psd_start = start_time - 64
psd_end = start_time
ifos = ["H1", "L1"]

# 3. 定義下載函數
def get_data(detectors, start, end):
    tensors = []
    print(f"Downloading data for {start} to {end}...")
    for det in detectors:
        ts = TimeSeries.fetch_open_data(det, start, end, verbose=False)
        ts = ts.resample(sample_rate)
        if np.isnan(ts.value).any():
            ts.value = np.nan_to_num(ts.value)
        tensors.append(torch.from_numpy(ts.value.copy()).float())
    return torch.stack(tensors).to(device)

# 4. 執行下載與處理
try:
    data_tensor = get_data(ifos, start_time, end_time).unsqueeze(0)
    psd_tensor = get_data(ifos, psd_start, psd_end)

    # 設定白化參數
    fduration = 2 # 濾波長度

    spectral_density = SpectralDensity(
        sample_rate=sample_rate,
        fftlength=2,
        average="median",
    ).to(device)

    whiten = Whiten(
        fduration=fduration,
        sample_rate=sample_rate,
        highpass=20
    ).to(device)

    psd = spectral_density(psd_tensor.double())
    whitened_data = whiten(data_tensor, psd)

    # === 關鍵：計算白化後的實際秒數 ===
    # Whiten 會切掉頭尾各 fduration/2
    actual_duration = raw_duration - fduration

    print("✅ 數據白化完成！")
    print(f"Whitened Data Shape: {whitened_data.shape}")
    print(f"Actual Duration for Q-transform: {actual_duration} seconds")

except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
import matplotlib.pyplot as plt
from ml4gw.transforms import SingleQTransform

# === 1. 計算 Spectrogram ===
q_transform = SingleQTransform(
    duration=actual_duration, # <--- 修正這裡！使用 6 秒而不是 8 秒
    sample_rate=sample_rate,
    spectrogram_shape=(512, 1024),
    q=10,
    frange=(20, sample_rate/2)
).to(device)

specgram = q_transform(whitened_data)
spec_np = specgram[0, 1].cpu().numpy() # 取 L1

# === 2. 牛頓力學擬合 ===
G = 6.674e-11
c = 3e8
M_solar = 1.989e30
m1 = 85 * M_solar
m2 = 66 * M_solar
chirp_mass = (m1 * m2)**(3/5) / (m1 + m2)**(1/5)

def newtonian_frequency(t, t_c):
    tau = t_c - t
    with np.errstate(invalid='ignore'):
        val = (c**3) / (8 * np.pi * G * chirp_mass) * \
              ((5 * G * chirp_mass) / (c**3 * tau))**(3/8)
    return val

# === 3. 繪圖 ===
# 時間軸也要修正，因為頭尾各被切掉了 fduration/2 (即 1秒)
plot_start = start_time + (fduration / 2)
plot_end = end_time - (fduration / 2)

plt.figure(figsize=(12, 8))
plt.imshow(spec_np, aspect="auto", origin="lower",
           extent=[plot_start, plot_end, 20, sample_rate/2],
           cmap='viridis', vmin=0, vmax=25)

plt.yscale('log')
plt.ylim(20, 150)
plt.xlim(trigger_time - 0.5, trigger_time + 0.2)

t_vals = np.linspace(trigger_time - 1, trigger_time, 500)
f_vals = newtonian_frequency(t_vals, trigger_time + 0.05)
plt.plot(t_vals, f_vals, 'r--', linewidth=2.5, label='Newtonian Prediction')

plt.xlabel("GPS Time (s)")
plt.ylabel("Frequency (Hz)")
plt.colorbar(label="Normalized Energy")
# 修正 SyntaxWarning: 使用 r"" raw string
plt.title(rf"GW190521: Spectrogram vs Newtonian Fit (Total Mass ~150 $M_\odot$)")
plt.legend()
plt.show()